In [38]:
# Finding and extracting the dataset
from os import walk
import zipfile

dataset_folder = './dataset'
    
filenames = next(walk(dataset_folder), (_, _, []))[2]
zip_filename = [filename for filename in filenames if filename.find('.zip') > -1][0]
file_path = dataset_folder + '/' + zip_filename

file_path

'./dataset/ds339_tx_2021_0913_091153.zip'

In [24]:
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_folder)

In [39]:
# Reading the dataset
import pandas as pd

filenames = next(walk(dataset_folder), (_, _, []))[2]
dataset_filename = [filename for filename in filenames if filename.find('.txt') > -1][0]
dataset_path = dataset_folder + '/' + dataset_filename

dataset_path

'./dataset/ds339_tx_All_Data_1059_2015_0729_215742.txt'

In [41]:
df = pd.read_csv(dataset_path, delimiter = "\t")

df.head()

,Row,Sample Name,Transaction Id,Anon Student Id,Session Id,Time,Time Zone,Duration (sec),Student Response Type,Student Response Subtype,...,KC (MCAS5-State_WPI-Simple),KC Category (MCAS5-State_WPI-Simple),KC (MCAS5-State_WPI-Simple).1,KC Category (MCAS5-State_WPI-Simple).1,KC (Single-KC),KC Category (Single-KC),KC (Unique-step),KC Category (Unique-step),School,Class
0,1,All Data,13cb29890fa2be0ca31bcd07f35000d6,Stu_000271be877ea1e7bf4f038c96cee5f9,5660580,2006-10-13 11:53:10,US/Eastern,22,ATTEMPT,NaN,...,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,ForestGrove,Period40607Dumphy
1,2,All Data,3fedf6c06ab6b9b3c36e245335445bcf,Stu_000271be877ea1e7bf4f038c96cee5f9,5660580,2006-10-13 11:53:13,US/Eastern,3,HINT_REQUEST,NaN,...,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,ForestGrove,Period40607Dumphy
2,3,All Data,0f5bb1a5c133ac1aa0a3d32e4d44dde9,Stu_000271be877ea1e7bf4f038c96cee5f9,5660580,2006-10-13 11:53:20,US/Eastern,7,HINT_REQUEST,NaN,...,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,ForestGrove,Period40607Dumphy
3,4,All Data,19a9aabf614d6f27dc9970dc842da081,Stu_000271be877ea1e7bf4f038c96cee5f9,5660580,2006-10-13 11:53:22,US/Eastern,2,HINT_REQUEST,NaN,...,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,ForestGrove,Period40607Dumphy
4,5,All Data,eae2bd81fd19ec6093c645d90e92222e,Stu_000271be877ea1e7bf4f038c96cee5f9,5660580,2006-10-13 11:53:32,US/Eastern,10,ATTEMPT,NaN,...,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,ForestGrove,Period40607Dumphy


In [43]:
df.describe()

,Row,Session Id,Student Response Subtype,Tutor Response Subtype,Problem Name,Problem View,Attempt At Step,Is Last Attempt,Help Level,Total Num Hints,...,KC Category (WPI-Apr-2005).4,KC Category (WPI-Apr-2005).5,KC Category (MCAS39-State_WPI-Simple),KC Category (MCAS39-State_WPI-Simple).1,KC Category (MCAS39-State_WPI-Simple).2,KC Category (MCAS39-State_WPI-Simple).3,KC Category (MCAS5-State_WPI-Simple),KC Category (MCAS5-State_WPI-Simple).1,KC Category (Single-KC),KC Category (Unique-step)
count,1.451003e+06,1.451003e+06,0.0,0.0,1.451003e+06,1.450822e+06,1.448076e+06,1.448076e+06,394514.000000,303756.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,7.255020e+05,4.409396e+06,NaN,NaN,5.609072e+03,1.281691e+00,2.341389e+00,4.472597e-01,2.010413,3.529932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,4.188686e+05,2.780981e+06,NaN,NaN,5.119680e+03,6.858265e-01,2.543439e+00,4.972108e-01,1.419201,1.298585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000e+00,6.720000e+02,NaN,NaN,7.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,3.627515e+05,1.363892e+06,NaN,NaN,1.149000e+03,1.000000e+00,1.000000e+00,0.000000e+00,1.000000,3.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,7.255020e+05,5.450824e+06,NaN,NaN,3.133000e+03,1.000000e+00,1.000000e+00,0.000000e+00,2.000000,3.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.088252e+06,6.838668e+06,NaN,NaN,9.562000e+03,1.000000e+00,3.000000e+00,1.000000e+00,3.000000,4.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.451003e+06,8.167924e+06,NaN,NaN,1.865100e+04,2.400000e+01,1.200000e+02,1.000000e+00,57.000000,47.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Drop unnecessary columns
df.nunique()

Row                                        1451003
Sample Name                                      1
Transaction Id                             1451003
Anon Student Id                               5046
Session Id                                  392886
Time                                       1027591
Time Zone                                        1
Duration (sec)                                 641
Student Response Type                            2
Student Response Subtype                         0
Tutor Response Type                              2
Tutor Response Subtype                           0
Level (Curriculum)                               1
Problem Name                                  1798
Problem View                                    17
Problem Start Time                          347840
Step Name                                     5834
Attempt At Step                                120
Is Last Attempt                                  2
Outcome                        